Special Thanks to:Gabriel Atkin

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression

In [ ]:
 rides_df =pd.read_csv('/kaggle/input/uber-lyft-cab-prices/cab_rides.csv')

In [ ]:
weather_df = pd.read_csv('/kaggle/input/uber-lyft-cab-prices/weather.csv')

In [ ]:
rides_df

In [ ]:
weather_df

In [ ]:
rides_df.info()

* It seems that we should drop price because it hasnt value for us. 
* We have a wide range of missing values

In [ ]:
weather_df.info()

* The same as price in ride table, we should drop rain column too
* Its full of NaN value

# Lets to cleaning our data

### Cleaning Ride Data

In [ ]:
rides_df.isna().sum()

In [ ]:
##Adding <br> to go to the next line

We find out the total number of our missing values= 55095<br>
It means 55095 example have no price so we should drop this columns<br> This is the only columns with missing values

In [ ]:
# Dropna: drop any missing values in columns or rows
ride_df=rides_df.dropna(axis=0).reset_index(drop=True)
ride_df

# Lets go to cleaning weather data

In [ ]:
weather_df.isna().sum()

In [ ]:
weather_df=weather_df.fillna(0)

# Creating Average weather DataFrame 

In [ ]:
weather_df

In [ ]:
weather_df.groupby('location').mean()
#weather_df()

In [ ]:
avr_weather_df= weather_df.groupby('location').mean().reset_index(drop=False)
avr_weather_df

In [ ]:
avr_weather_df = avr_weather_df.drop('time_stamp', axis=1)
avr_weather_df

# Marging DataFrame

In [ ]:
rides_df

In [ ]:
source_weather_df= avr_weather_df.rename(
columns={
    'location':'source',
    'temp':'source_temp',
    'clouds': 'source_clouds',
    'pressure':'source_pressure',
    'rain': 'source_rain',
    'humidity':'source_humidity',
    'wind':'source_wind'
}
)
source_weather_df

In [ ]:
destination_weather_df= avr_weather_df.rename(
columns={
    'location':'destination',
    'temp':'destination_temp',
    'clouds': 'destination_clouds',
    'pressure':'destination_pressure',
    'rain': 'destination_rain',
    'humidity':'destination_humidity',
    'wind':'destination_wind'
}
)
destination_weather_df

In [ ]:
rides_df.merge(source_weather_df,on='source')

In [ ]:
data = rides_df\
    .merge(source_weather_df, on ='source')\
    .merge(destination_weather_df, on='destination')

data

## Preprocessing

In [ ]:
def onehot_encode(df, column, prefix):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=prefix)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [ ]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop id column
    df = df.drop('id', axis=1)
    
    # Binary encode cab_type column
    df['cab_type'] = df['cab_type'].replace({'Lyft': 0, 'Uber': 1})
    
    # One-hot encode remaining categorical columns
    for column, prefix in [('destination', "dest"), ('source', "src"), ('product_id', "pid"), ('name', "nm")]:
        df = onehot_encode(df, column=column, prefix=prefix)
    
    # Split df into X and y
    y = df['price']
    X = df.drop('price', axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

In [ ]:
X_train

In [ ]:
y_train

## Training

In [ ]:
# Import necessary libraries
from sklearn.impute import SimpleImputer

# Initialize the SimpleImputer for X_train
imputer_X = SimpleImputer(strategy='mean')

# Fit and transform X_train
X_train_imputed = imputer_X.fit_transform(X_train)

# Initialize the SimpleImputer for y_train
imputer_y = SimpleImputer(strategy='mean')

# Reshape y_train if necessary
y_train_reshaped = y_train.values.reshape(-1, 1)

# Fit and transform y_train
y_train_imputed = imputer_y.fit_transform(y_train_reshaped)

# Optionally, set the feature names for X_train to suppress the warning
imputer_X.feature_names_in_ = X_train.columns

# Ensure the output is in the desired shape
print("Shape of X_train_imputed:", X_train_imputed.shape)
print("Shape of y_train_imputed:", y_train_imputed.shape)


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

imputer = SimpleImputer(strategy='mean')


model = LinearRegression()

model.fit(X_train_imputed, y_train_imputed)

imputer.fit(X_train)
imputer.feature_names_in_ = None

# Reshape X_test to have the same number of features as X_train
X_test_reshaped = X_test.values.reshape(-1, X_train.shape[1])

# Transform X_test_reshaped using the fitted imputer
X_test_imputed = imputer.transform(X_test_reshaped)

# X_test_reshaped = X_test.values.reshape(-1, 1)
# X_test_imputed = imputer.transform(X_test_reshaped)

y_pred = model.predict(X_test_imputed)
y_pred[0]


In [ ]:
# Reshape y_train to have a single feature
y_train_reshaped = y_train.values.reshape(-1, 1)

# Fit the imputer to the reshaped y_train
imputer.fit(y_train_reshaped)

# Since y_train is 1D, there are no feature names to set
imputer.feature_names_in_ = None

# Now, you can transform the test data in a similar manner
y_test_reshaped = y_test.values.reshape(-1, 1)
y_test_imputed = imputer.transform(y_test_reshaped)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Make predictions on the test data
y_pred = model.predict(X_test_imputed)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test_imputed, y_pred)

# Calculate R^2 score
r2 = r2_score(y_test_imputed, y_pred)

print("Mean Squared Error (MSE):", mse)
print("R^2 Score:", r2)

In [ ]:
import pickle

# Save the trained model to a file
with open('linear_regression_model.pkl', 'wb') as file:
    pickle.dump(model, file)
